In [1]:
import pickle

Load required objects

In [2]:
with open('complete_data_2021.pkl', 'rb') as f:
    complete_data_2021 = pickle.load(f)
with open('complete_data_scaler.pkl', 'rb') as f:
    complete_data_scaler = pickle.load(f)
with open('important_features.pkl', 'rb') as f:
    important_features = pickle.load(f)    

Load the ML model

In [3]:
from tensorflow.keras.models import load_model
nn_model = load_model('nn_model.h5')

Predict with user specified values

In [4]:
complete_data_2021

avg_temp_val_1                          50.2
avg_temp_val_2                          51.8
avg_temp_val_3                          51.9
avg_temp_val_4                          58.1
avg_temp_val_5                          63.1
                                        ... 
('storm_count', 'Heavy Snow')            0.0
('storm_count', 'Thunderstorm Wind')     0.0
('storm_count', 'Tornado')               0.0
('storm_count', 'Wildfire')              1.0
('storm_count', 'Winter Weather')        0.0
Name: 2021, Length: 180, dtype: float64

**This is how the function works:**

We start with the assumption that we have a list of features with new values (from the user input sliders)

In [6]:
test_user_input = {'avg_temp_val_1': 99, 'avg_temp_val_2': 99, }

We then replace these feature values one-by-one

In [7]:
for k, v in test_user_input.items():
    complete_data_2021[k] = v

Below, we can see that the values for `avg_temp_val_1` and `avg_temp_val_2` were replaced.

In [8]:
complete_data_2021

avg_temp_val_1                          99.0
avg_temp_val_2                          99.0
avg_temp_val_3                          51.9
avg_temp_val_4                          58.1
avg_temp_val_5                          63.1
                                        ... 
('storm_count', 'Heavy Snow')            0.0
('storm_count', 'Thunderstorm Wind')     0.0
('storm_count', 'Tornado')               0.0
('storm_count', 'Wildfire')              1.0
('storm_count', 'Winter Weather')        0.0
Name: 2021, Length: 180, dtype: float64

Now, we scale the values for input to the ML model.

In [15]:
complete_data_scaler.transform(complete_data_2021.values.reshape(1, -1))

/Users/spartan/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[6.79746835e+00, 5.55789474e+00, 3.27102804e-01, 7.50000000e-01,
        6.10619469e-01, 9.04761905e-01, 7.39130435e-01, 6.31578947e-01,
        6.44736842e-01, 1.12244898e-01, 8.24074074e-01, 4.40000000e-01,
        6.20253165e-01, 5.89473684e-01, 3.27102804e-01, 7.50000000e-01,
        6.10619469e-01, 9.04761905e-01, 7.39130435e-01, 6.31578947e-01,
        6.44736842e-01, 1.12244898e-01, 8.24074074e-01, 4.40000000e-01,
        5.46218487e-01, 6.72131148e-01, 4.41379310e-01, 8.47619048e-01,
        6.91011236e-01, 8.71287129e-01, 7.15909091e-01, 6.62921348e-01,
        7.19512195e-01, 1.11111111e-01, 6.89393939e-01, 2.53164557e-01,
        5.46218487e-01, 6.72131148e-01, 4.41379310e-01, 8.47619048e-01,
        6.91011236e-01, 8.71287129e-01, 7.15909091e-01, 6.62921348e-01,
        7.19512195e-01, 1.11111111e-01, 6.89393939e-01, 2.53164557e-01,
        5.43689320e-01, 5.12195122e-01, 7.04225352e-02, 5.24590164e-01,
        4.86111111e-01, 8.67924528e-01, 8.00000000e-01, 6.285714

We now predict the electricity consumption using NN model. Note the use of y_max. This is the maximum electricity consumption in the dataset. Since I am using the softmax layer as output in my NN model, the output is always between 0 and 1. The value then needs to be scaled back to the original value. We can achieve this by multiplying the predicted value with y_max.

In [22]:
# from the complete dataset
y_max = 3308.467553

def predict_consumption(feature_dict):
    x = complete_data_2021.copy()
    for k, v in feature_dict.items():
        x[k] = v
    scaled_x = complete_data_scaler.transform(x.values.reshape(1, -1))
    return nn_model.predict(scaled_x)[0][0] * y_max
    

In [23]:
predict_consumption(test_user_input)

1/1 [==============================] - 0s 16ms/step


/Users/spartan/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


3080.7307002040916